## Autogen Agents

*[Coding along with the Udemy online course AI Agents: Building Teams of LLM Agents that Work For You by Mohsen Hassan & Ilyass Tabiai]*

**AutoGen is an open-source framework that leverages multiple agents to enable complex workflows.**

- Autogen was developed by Microsoft to create advanced LLM-based applications.
- Autogen is designed to enable you to orchestrate multi-agent conversations that seamlessly integrate LLMs, human input and other tools.
- Autogen is supported by Microsoft, which gives it long term viability and was designed to specifically work with OpenAI, so it will be the framework we will be using in this class for agentic design.

**An online tutorial can be found at [Getting Started with AutoGen](https://microsoft.github.io/autogen/docs/Getting-Started).**

In [1]:
from autogen import ConversableAgent

Importing `autogen` returned the following error:<br/>
`flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.`

The temporary solution to solve this was to create a constraints.txt file with the following content:

`flaml==2.2.0`

Then running: `pip install -c constraints.txt autogen`

Following the suggestion at https://github.com/microsoft/autogen/issues/3548.

*FLAML @ https://microsoft.github.io/FLAML/ is "A Fast Library for Automated Machine Learning & Tuning."*

<img decoding="async" loading="lazy" alt="AutoGen Overview" src="../assets/images/microsoft-autogen-1.png" width="75%">
<br/><i>Screenshot from Getting Started with AutoGen @ https://microsoft.github.io/autogen/docs/Getting-Started</i>

## Conversable Agent

* AutoGen offers a unified multi-agent conversation framework that simplifies the orchestration and automation of complex LLM workflows the Conversable Agent class. These agents can collectively perform tasks autonomously or with human feedback.
* Conversable Agents are:
    * **Conversable**: Agents can send and receive messages from other agents or humans; they can initate and continue a conversation seamlessly.
    * **Customizable**: They can be customized to integrate LLMs, human intervention, tools/skills (code) or combinations of these.
    * Of two subcategories:
        * **AssistantAgent**: This subcategory represents AI assistants, that use LLMs (such as GPT-4) by default. It can generate Python code for users to execute based on task descriptions. Additionally, it can receive execution results (from code or other agents) and suggest corrections or bug fixes.
        * **UserProxyAgent**:  This subcategory acts as a proxy agent for humans. It solicits human input as its reply during interactions. It also has the capability to execute code and call functions or tools. Code execution can be automatically triggered when an executable code block is detected in the received message.

Let's create a Conversable Agent to accomlpish a simple task. Before doing that, we will have to create a LLM config that specifies which LLM we want to use. Here we will again use chatGPT3.5 for now.

In [2]:
from openai import OpenAI
import pandas as pd

In [3]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


### AssistantAgent Example

In [4]:
# creating a Conversable Agent to accomlpish a simple task
# 1st thing to do is to create a LLM config that specifies which LLM we want to use
# model from the list of models provided by OpenAI https://platform.openai.com/docs/models/continuous-model-upgrades
llm_config = {
    "model": "gpt-4o-mini",
    # "model": "gpt-3.5-turbo",
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [5]:
# configuring a Conversable Agent that will NEVER ask for our input and will use the defined model to answer
agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config, # The Agent will use the LLM config provided to answer
    human_input_mode="NEVER", # Can also be ALWAYS or TERMINATE (at end only)
)

In [6]:
agent

In [7]:
# let's send a first request (= a prompt) to this agent
# using the `generate_reply()` function of this agent
# the message gets sent as a dictionary that must specify the `content` and the `role` keys
# (we could actually send several messages in ión request) 
# the `user` role means this is a request, we could use
# the `system` role to specify a system prompt for this agent
reply = agent.generate_reply(
    messages=[
        {
            "content": "Tell me a fun fact about money.",
            "role": "user"
        }
    ]
)
reply

'A fun fact about money is that the first paper money was created in China during the Tang Dynasty, around the 7th century. However, it was during the Song Dynasty (11th century) that it became widely used. This paper currency was called "jiaozi" and was initially backed by precious metals, making it a revolutionary development in commerce and trade at the time!'

In [8]:
print(reply)

A fun fact about money is that the first paper money was created in China during the Tang Dynasty, around the 7th century. However, it was during the Song Dynasty (11th century) that it became widely used. This paper currency was called "jiaozi" and was initially backed by precious metals, making it a revolutionary development in commerce and trade at the time!


__Important to note point: when we use this `generate_reply()` function, we do not alter the state of this agent. This doesn't allow to have long conversations about a subject or solve tasks that require several steps. To explain what this means: if we try to ask this agent something that refers to this last query, we'll see that it has no idea what was this previous query.__

In [9]:
reply = agent.generate_reply(
    messages=[
        {
            "content": "Repeat the fact.", 
            "role": "user"
        }
    ]
)
print(reply)

Sure! Please provide the fact you'd like me to repeat.


__To solve this we need to keep the state to be able to solve complex tasks. So we first need to be able to create conversations that alter the state of an agent if we want to solve complex tasks. An approach to achieve this is to create two agents that have evolving states based on their answers and that will converse based on these states.__